# Extraction of Spindles from Neocortex recordings

Restarting from LFPwake0 and LFPwakeremoved.

LFPwakeremoved will be used to determined signal variance for threshold adjustement. 

LFPwake0 will be used for time determination. 

## Load LFP and packages

In [152]:
cd "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/"

\\10.69.168.1\crnldata\waking\audrey_hay\L1imaging\AnalysedMarch2023


In [153]:
from scipy import signal
from scipy import stats
from scipy.signal import find_peaks, peak_widths
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
import os
%matplotlib widget

folder_base = Path('Gaelle/Baseline_recording/GreenLinesOK/session5/OpenEphys') 

filename = folder_base / f'LFPwake0.npy'
filename3 = folder_base / f'LFPwakeremoved.npy'
filename2 = folder_base / f'RawDataChannelExtractedDS.npy'
EMGbooleaninput = folder_base / f'EMGframeBoolean.pkl'
Channels = 'LFPChannels_perMice.xlsx' 

EMGboolean = pd.read_pickle(EMGbooleaninput)
LFPwakeremoved = np.load(filename3, mmap_mode= 'r')
All = np.load(filename2, mmap_mode= 'r')

def readfile(file):
  with open(file, encoding="utf-8") as file:
    list = []
    for row in file:
      temp = row.split()
      list.append(temp[:])
    return list

mice = os.path.basename(os.path.dirname(os.path.dirname(folder_base)))
allchannels = pd.read_excel(Channels)
PFCch1=int(allchannels[mice][0].split(',')[0])
PFCch2=int(allchannels[mice][0].split(',')[1])
S1ch1=int(allchannels[mice][1].split(',')[0])
S1ch2=int(allchannels[mice][1].split(',')[1])

PFC  =  All[:, PFCch1]-All[:, PFCch2] 
PFCwakeremoved = LFPwakeremoved[:,PFCch1]-LFPwakeremoved[:,PFCch2] 

S1  =  All[:, S1ch1]-All[:, S1ch2] 
S1wakeremoved = LFPwakeremoved[:,S1ch1]-LFPwakeremoved[:,S1ch2] 

## Continuous Wavelet Transform and projection calculation

First on signal with no wake time to determine sd of signal

In [154]:
# Filter parameter :
f_lowcut = 10.
f_hicut = 16.
N = 4
fs = 1000
nyq = 0.5 * fs
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filter creation :
b, a = signal.butter(N, Wn, 'band')

filt_S1 = signal.filtfilt(b, a, S1)
filt_S1wakeremoved = signal.filtfilt(b, a, S1wakeremoved)

filt_PFC = signal.filtfilt(b, a, PFC)
filt_PFCwakeremoved = signal.filtfilt(b, a, PFCwakeremoved)


In [155]:
# Parameter and computation of CWT
w = 10.
freq = np.linspace(10, 16, 6)#18)
widths = w*fs / (2*freq*np.pi)
PFCNWcwt = signal.cwt(filt_PFCwakeremoved, signal.morlet2, widths, w=w)
S1NWcwt = signal.cwt(filt_S1wakeremoved, signal.morlet2, widths, w=w)

# Projection calculation PFC
absPFCNWcwt = np.absolute(PFCNWcwt)
proj_PFCNWcwt = np.sum(absPFCNWcwt, axis = 0)/24
sdproj_PFCcwt = np.std(proj_PFCNWcwt)
sd6proj_PFCcwt = sdproj_PFCcwt*6
sd7proj_PFCcwt = sdproj_PFCcwt*7

# Projection calculation S1
absS1NWcwt = np.absolute(S1NWcwt)
proj_S1NWcwt = np.sum(absS1NWcwt, axis = 0)/24
sdproj_S1cwt = np.std(proj_S1NWcwt)
sd6proj_S1cwt = sdproj_S1cwt*6
sd7proj_S1cwt = sdproj_S1cwt*7

Second on the signal for which wake times have been zeroed

In [156]:
#####################################
########         PFC         #########
#####################################
# Conservative boolean filtering of PFC filtered signal
BooleanCons = EMGboolean['BooleanConservative']
fPFCwake0C = filt_PFC.copy()
fPFCwake0C[BooleanCons] = 0
PFCwake0C = PFC.copy()
PFCwake0C[BooleanCons] = 0
# Liberal boolean filtering of PFC filtered signal
BooleanLib = EMGboolean['BooleanLiberal']
fPFCwake0L = filt_PFC.copy()
fPFCwake0L[BooleanLib] = 0
PFCwake0L = PFC.copy()
PFCwake0L[BooleanLib] = 0

# Computation of CWT
PFCcwtWake0cons = signal.cwt(fPFCwake0C, signal.morlet2, widths, w=w)
PFCcwtWake0lib = signal.cwt(fPFCwake0L, signal.morlet2, widths, w=w)

# Projection calculation
absPFCW0Ccwt = np.absolute(PFCcwtWake0cons)
proj_PFCW0Ccwt = np.sum(absPFCW0Ccwt, axis = 0)/24
absPFCW0Lcwt = np.absolute(PFCcwtWake0lib)
proj_PFCW0Lcwt = np.sum(absPFCW0Lcwt, axis = 0)/24


#####################################
########         S1         #########
#####################################
# Conservative boolean filtering of S1 filtered signal
BooleanCons = EMGboolean['BooleanConservative']
fS1wake0C = filt_S1.copy()
fS1wake0C[BooleanCons] = 0
S1wake0C = S1.copy()
S1wake0C[BooleanCons] = 0
# Liberal boolean filtering of S1 filtered signal
BooleanLib = EMGboolean['BooleanLiberal']
fS1wake0L = filt_S1.copy()
fS1wake0L[BooleanLib] = 0
S1wake0L = S1.copy()
S1wake0L[BooleanLib] = 0

# Computation of CWT
S1cwtWake0cons = signal.cwt(fS1wake0C, signal.morlet2, widths, w=w)
S1cwtWake0lib = signal.cwt(fS1wake0L, signal.morlet2, widths, w=w)

# Projection calculation
absS1W0Ccwt = np.absolute(S1cwtWake0cons)
proj_S1W0Ccwt = np.sum(absS1W0Ccwt, axis = 0)/24
absS1W0Lcwt = np.absolute(S1cwtWake0lib)
proj_S1W0Lcwt = np.sum(absS1W0Lcwt, axis = 0)/24

## Extracting Spindles and determining main properties 

First extraction of spindle peaks, initiation, end and width

In [157]:
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

# 7 sd threshold
peaks, properties = find_peaks(proj_PFCW0Lcwt, width=200, height=sd7proj_PFCcwt)
properties["prominences"], properties["widths"]

# Spindles boundaries taken at 70% from peak of intensity. This means that the spindles with small amplitude will be longer than the big ones.
results_width = peak_widths(proj_PFCW0Lcwt, peaks, rel_height=0.7)

# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
Spindle_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

Second extraction of main frequency and power 

In [158]:
projMaxP_cwtmg = np.max(PFCcwtWake0lib, axis = 0)
projMaxF_cwtmg = np.argmax(PFCcwtWake0lib, axis = 0)+ 10 #/2 + 8
projMaxP_cwtmg.shape

nb_Spindles = np.arange(0,len(peaks),1)
data = np.zeros((len(peaks),4))

for tt in nb_Spindles:
    Spindle_start = int(Spindle_prop[3,tt])
    Spindle_stop = int(Spindle_prop[4,tt])
    Spindle_MaxP = projMaxP_cwtmg[Spindle_start:Spindle_stop]
    Spindle_MaxF = projMaxF_cwtmg[Spindle_start:Spindle_stop]
    data[tt, 0] = max(Spindle_MaxF).round()
    data[tt, 1] = max(Spindle_MaxP).round()
    data[tt, 2] = round(sum(Spindle_MaxF)/len(Spindle_MaxF))
    data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))

param_Spindle = pd.DataFrame(data, columns = ['Max freq', 'Max int', 'Avg freq', 'Avg int'])
tSpindle_prop = Spindle_prop.transpose()
pd_prop_Spindle = pd.DataFrame(tSpindle_prop, columns = ['peak time', 'Duration', 'peak amp', 'start time', 'end time'])
All_Spindle = pd.concat([pd_prop_Spindle, param_Spindle], axis=1)

C:\Users\Manip2\AppData\Local\Temp\ipykernel_13360\3926384638.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 1] = max(Spindle_MaxP).round()
C:\Users\Manip2\AppData\Local\Temp\ipykernel_13360\3926384638.py:16: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))
C:\Users\Manip2\AppData\Local\Temp\ipykernel_13360\3926384638.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))


In [159]:
nb_spindle = All_Spindle.shape[0]
listtodrop = []
for tt in range(nb_spindle-1):
    if(All_Spindle['end time'][tt]>All_Spindle['start time'][tt + 1]):
        if(All_Spindle['Duration'][tt]<All_Spindle['Duration'][tt + 1]):
            if(All_Spindle['start time'][tt]<All_Spindle['start time'][tt + 1]):
                All_Spindle['start time'][tt+1] = All_Spindle['start time'][tt]
                listtodrop.append(tt)
            else:
                listtodrop.append(tt)
        if(All_Spindle['Duration'][tt]>All_Spindle['Duration'][tt + 1]):
            if(All_Spindle['end time'][tt]<All_Spindle['end time'][tt + 1]):
                All_Spindle['end time'][tt] = All_Spindle['end time'][tt + 1]
                listtodrop.append(tt+1)
            else:
                listtodrop.append(tt+1)

for tt in range(nb_spindle-1):
    if((All_Spindle['start time'][tt + 1] - All_Spindle['end time'][tt])<200):
        if((All_Spindle['Duration'][tt]+300)<All_Spindle['Duration'][tt + 1]):
            All_Spindle['start time'][tt + 1] = All_Spindle['start time'][tt]
            listtodrop.append(tt)
        if((All_Spindle['Duration'][tt+1]+300)<All_Spindle['Duration'][tt]):
            All_Spindle['end time'][tt] = All_Spindle['start time'][tt + 1]
            listtodrop.append(tt+1)

for tt in range(nb_spindle):
    All_Spindle['Duration'][tt]=All_Spindle['end time'][tt]-All_Spindle['start time'][tt]
All_Spindle = All_Spindle.drop(listtodrop) 
#print(listtodrop)
All_Spindle.shape[0]

11

### Store the results in All_Spindle_prop pd dataframe and save as pkl/csv for post processing.

End of Notebook. 

In [160]:
filename2 = folder_base / f'Spindlesproperties_PFCInitial_AB.pkl'
filename3 = folder_base / f'Spindlesproperties_PFCInitial_AB.csv'
All_Spindle.to_pickle(filename2)
All_Spindle.to_csv(filename3, sep = ',')

filename2 = folder_base / f'Spindlesproperties_PFC_AB.pkl'
filename3 = folder_base / f'Spindlesproperties_PFC_AB.csv'
All_Spindle.to_pickle(filename2)
All_Spindle.to_csv(filename3, sep = ',')

### Display

ephys viewer to check Spindle detection

In [161]:
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer

app = mkQApp()

sample_rate = 1000.
t_start = 0.

combined = np.stack([PFC, filt_PFC, proj_PFCW0Ccwt, proj_PFCW0Lcwt], axis = 1)

Spindle_peak = peaks
Spindle_start = Spindle_prop[3,:].astype(int)
Spindle_end = Spindle_prop[4,:].astype(int)
Spindle_peak = All_Spindle['peak time'].to_numpy().astype(int)
Spindle_start = All_Spindle['start time'].to_numpy().astype(int)
Spindle_end = All_Spindle['end time'].to_numpy().astype(int)
#Spindle_start = All_Spindle[3,:].astype(int)
#Spindle_end = All_Spindle[4,:].astype(int)

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: Spindle_peak, 1: Spindle_start, 2: Spindle_end}
#and asign them to some channels each
scatter_channels = {0: [1, 2], 1: [0, 1], 2: [0, 1]}
source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'
view1.by_channel_params['ch3', 'color'] = '#ffffff'

view1.by_channel_params['ch0', 'gain'] = 0.0003
view1.by_channel_params['ch1', 'gain'] = 0.001
view1.by_channel_params['ch2', 'gain'] = 0.001
view1.by_channel_params['ch3', 'gain'] = 0.001


view1.by_channel_params['ch0', 'offset'] = 1
view1.by_channel_params['ch1', 'offset'] = -1
view1.by_channel_params['ch2', 'offset'] = -1
view1.by_channel_params['ch3', 'offset'] = -1


#put this viewer in the main window
win.add_view(view1)

#Run
win.show()
app.exec_()

debug True
QT_MODE PyQt5
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0009975433349609375 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0006771087646484375 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.005027055740356445 s
refresh duration for  0.0 

0

In [162]:
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

# 7 sd threshold
peaks, properties = find_peaks(proj_S1W0Lcwt, width=200, height=sd6proj_S1cwt)
properties["prominences"], properties["widths"]

# Spindles boundaries taken at 70% from peak of intensity. This means that the spindles with small amplitude will be longer than the big ones.
results_width = peak_widths(proj_S1W0Lcwt, peaks, rel_height=0.7)

# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
Spindle_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

In [163]:
projMaxP_cwtmg = np.max(S1cwtWake0lib, axis = 0)
projMaxF_cwtmg = np.argmax(S1cwtWake0lib, axis = 0)+ 10 #/2 + 8

nb_Spindles = np.arange(0,len(peaks),1)
data = np.zeros((len(peaks),4))

for tt in nb_Spindles:
    Spindle_start = int(Spindle_prop[3,tt])
    Spindle_stop = int(Spindle_prop[4,tt])
    Spindle_MaxP = projMaxP_cwtmg[Spindle_start:Spindle_stop]
    Spindle_MaxF = projMaxF_cwtmg[Spindle_start:Spindle_stop]
    data[tt, 0] = max(Spindle_MaxF).round()
    data[tt, 1] = max(Spindle_MaxP).round()
    data[tt, 2] = round(sum(Spindle_MaxF)/len(Spindle_MaxF))
    data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))

param_Spindle = pd.DataFrame(data, columns = ['Max freq', 'Max int', 'Avg freq', 'Avg int'])
tSpindle_prop = Spindle_prop.transpose()
pd_prop_Spindle = pd.DataFrame(tSpindle_prop, columns = ['peak time', 'Duration', 'peak amp', 'start time', 'end time'])
All_Spindle = pd.concat([pd_prop_Spindle, param_Spindle], axis=1)

C:\Users\Manip2\AppData\Local\Temp\ipykernel_13360\1716516439.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 1] = max(Spindle_MaxP).round()
C:\Users\Manip2\AppData\Local\Temp\ipykernel_13360\1716516439.py:15: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))
C:\Users\Manip2\AppData\Local\Temp\ipykernel_13360\1716516439.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))


In [164]:
nb_spindle = All_Spindle.shape[0]
listtodrop = []
for tt in range(nb_spindle-1):
    if(All_Spindle['end time'][tt]>All_Spindle['start time'][tt + 1]):
        if(All_Spindle['Duration'][tt]<All_Spindle['Duration'][tt + 1]):
            if(All_Spindle['start time'][tt]<All_Spindle['start time'][tt + 1]):
                All_Spindle['start time'][tt+1] = All_Spindle['start time'][tt]
                listtodrop.append(tt)
            else:
                listtodrop.append(tt)
        if(All_Spindle['Duration'][tt]>All_Spindle['Duration'][tt + 1]):
            if(All_Spindle['end time'][tt]<All_Spindle['end time'][tt + 1]):
                All_Spindle['end time'][tt] = All_Spindle['end time'][tt + 1]
                listtodrop.append(tt+1)
            else:
                listtodrop.append(tt+1)

for tt in range(nb_spindle-1):
    if((All_Spindle['start time'][tt + 1] - All_Spindle['end time'][tt])<200):
        if((All_Spindle['Duration'][tt]+300)<All_Spindle['Duration'][tt + 1]):
            All_Spindle['start time'][tt + 1] = All_Spindle['start time'][tt]
            listtodrop.append(tt)
        if((All_Spindle['Duration'][tt+1]+300)<All_Spindle['Duration'][tt]):
            All_Spindle['end time'][tt] = All_Spindle['start time'][tt + 1]
            listtodrop.append(tt+1)

for tt in range(nb_spindle):
    All_Spindle['Duration'][tt]=All_Spindle['end time'][tt]-All_Spindle['start time'][tt]
All_Spindle = All_Spindle.drop(listtodrop) 
All_Spindle.shape[0]

11

In [165]:
filename2 = folder_base / f'Spindlesproperties_S1Initial_AB.pkl'
filename3 = folder_base / f'Spindlesproperties_S1Initial_AB.csv'
All_Spindle.to_pickle(filename2)
All_Spindle.to_csv(filename3, sep = ',')

filename2 = folder_base / f'Spindlesproperties_S1_AB.pkl'
filename3 = folder_base / f'Spindlesproperties_S1_AB.csv'
All_Spindle.to_pickle(filename2)
All_Spindle.to_csv(filename3, sep = ',')

In [166]:
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer

app = mkQApp()

sample_rate = 1000.
t_start = 0.

combined = np.stack([S1, filt_S1, proj_S1W0Ccwt, proj_S1W0Lcwt], axis = 1)

Spindle_peak = peaks
Spindle_start = Spindle_prop[3,:].astype(int)
Spindle_end = Spindle_prop[4,:].astype(int)
Spindle_peak = All_Spindle['peak time'].to_numpy().astype(int)
Spindle_start = All_Spindle['start time'].to_numpy().astype(int)
Spindle_end = All_Spindle['end time'].to_numpy().astype(int)
#Spindle_start = All_Spindle[3,:].astype(int)
#Spindle_end = All_Spindle[4,:].astype(int)

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: Spindle_peak, 1: Spindle_start, 2: Spindle_end}
#and asign them to some channels each
scatter_channels = {0: [1, 2], 1: [0, 1], 2: [0, 1]}
source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'
view1.by_channel_params['ch3', 'color'] = '#ffffff'

view1.by_channel_params['ch0', 'gain'] = 0.0003
view1.by_channel_params['ch1', 'gain'] = 0.001
view1.by_channel_params['ch2', 'gain'] = 0.001
view1.by_channel_params['ch3', 'gain'] = 0.001


view1.by_channel_params['ch0', 'offset'] = 1
view1.by_channel_params['ch1', 'offset'] = -1
view1.by_channel_params['ch2', 'offset'] = -1
view1.by_channel_params['ch3', 'offset'] = -1


#put this viewer in the main window
win.add_view(view1)

#Run
win.show()
app.exec_()

debug True
QT_MODE PyQt5
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0010404586791992188 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.001008749008178711 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0009965896606445312 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.009804964065551758 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh d

0